| method | qubits | cnots | runtime |
| --- |:---:|:---:|:---:|
| isa | 10 | 1420.2 | 0.7174319744110107 |
| vqc | 10 | 51.0 | 83.52064890861512 |
| ucg | 10 | 2658.0 | 2.858856439590454 |


In [8]:
import numpy as np
import numpy as np
import os
from IPython.display import Markdown, display
from qiskit import QuantumCircuit, QuantumRegister, transpile
from qclib.state_preparation import UCGInitialize
from datetime import datetime
import time
import qc
import qiskit
n_qubits = 10
CX_ucg_counts = []
Runtimes_ucg = []

params = {'protein': 'homo_sapien', 'num_protein': 5, 'num_qubits':10, 'Avg_CX': 0, 'Avg_Runtime(s)': 0}

coupling_map = []
for i in range(n_qubits - 1):
    coupling_map.append([i, i + 1])
    coupling_map.append([i + 1, i])


params['num_protein'] = 10
for i in range(params['num_protein']):
    state_file = f"./States/state{i}.txt"
    state = []
    with open(state_file, 'r') as file:
        for line in file:
            columns = line.split()  
            if len(columns) >= 3:  

                amp = float(columns[1])  
                phase = float(columns[2])  
                if phase == 0.0:  
                    state.append(amp)
                else:
                    state.append(-1*amp)


        state = state/np.linalg.norm(state)
        ucg_time = time.time()

        # qclib ucg
        circuit_ucg = QuantumCircuit(n_qubits)
        UCGInitialize.initialize(circuit_ucg, state)



        circuits = [ ('ucg', circuit_ucg) ]



        transpiled = qiskit.transpile(circuit_ucg, basis_gates=['u', 'cx'], coupling_map=coupling_map, optimization_level=3)
        ucg_runtime = time.time() - ucg_time
        Runtimes_ucg.append(ucg_runtime)
        cx = transpiled.count_ops().get('cx', 0)
        CX_ucg_counts.append(cx)
print(CX_ucg_counts)

[2662, 2666, 2664, 2664, 2664, 2662, 2666, 2668, 2662, 2645]
